In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
tf.__version__

**Load the data**

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import struct
import matplotlib.pyplot as plt
path = '/kaggle/input/coic/train-images-idx3-ubyte'
with open(path,'rb') as f1:
    buf1 = f1.read() 
image_index = 0
image_index += struct.calcsize('>IIII')
temp = struct.unpack_from('>784B', buf1, image_index) 
# '>784B'的意思就是用大端法读取784( 28*28 )个unsigned byte
im = np.reshape(temp,(28,28))
plt.imshow(im , cmap='gray')

In [ ]:
####  https://www.jianshu.com/p/84f72791806f
import numpy as np
import struct
import matplotlib.pyplot as plt

train_images_idx3_ubyte_file = '/kaggle/input/coic/train-images-idx3-ubyte'
train_labels_idx1_ubyte_file = '/kaggle/input/coic/train-labels-idx1-ubyte'

test_images_idx3_ubyte_file = '/kaggle/input/coic/test-images-idx3-ubyte'
test_labels_idx1_ubyte_file = '/kaggle/input/coic/test-labels-idx1-ubyte'
final_test_idx3_ubyte_file = '/kaggle/input/coic/final-test-images-idx3-ubyte'



def decode_idx3_ubyte(idx3_ubyte_file):
    
    bin_data = open(idx3_ubyte_file, 'rb').read()

    offset = 0
    fmt_header = '>iiii'
    magic_number, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, bin_data, offset)

    image_size = num_rows * num_cols
    offset += struct.calcsize(fmt_header)
    fmt_image = '>' + str(image_size) + 'B'
    images = np.empty((num_images, num_rows, num_cols))
    for i in range(num_images):
        images[i] = np.array(struct.unpack_from(fmt_image, bin_data, offset)).reshape((num_rows, num_cols))
        offset += struct.calcsize(fmt_image)
    return images


def decode_idx1_ubyte(idx1_ubyte_file):

    bin_data = open(idx1_ubyte_file, 'rb').read()


    offset = 0
    fmt_header = '>ii'
    magic_number, num_images = struct.unpack_from(fmt_header, bin_data, offset)

    
    offset += struct.calcsize(fmt_header)
    fmt_image = '>B'
    labels = np.empty(num_images)
    for i in range(num_images):
        labels[i] = struct.unpack_from(fmt_image, bin_data, offset)[0]
        offset += struct.calcsize(fmt_image)
    return labels


def load_train_images(idx_ubyte_file=train_images_idx3_ubyte_file):
    """
    TRAINING SET IMAGE FILE (train-images-idx3-ubyte):
    [offset] [type]          [value]          [description]
    0000     32 bit integer  0x00000803(2051) magic number
    0004     32 bit integer  60000            number of images
    0008     32 bit integer  28               number of rows
    0012     32 bit integer  28               number of columns
    0016     unsigned byte   ??               pixel
    0017     unsigned byte   ??               pixel
    ........
    xxxx     unsigned byte   ??               pixel
    Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black).

    :param idx_ubyte_file: idx文件路径
    :return: n*row*col维np.array对象，n为图片数量
    """
    return decode_idx3_ubyte(idx_ubyte_file)


def load_train_labels(idx_ubyte_file=train_labels_idx1_ubyte_file):
    """
    TRAINING SET LABEL FILE (train-labels-idx1-ubyte):
    [offset] [type]          [value]          [description]
    0000     32 bit integer  0x00000801(2049) magic number (MSB first)
    0004     32 bit integer  60000            number of items
    0008     unsigned byte   ??               label
    0009     unsigned byte   ??               label
    ........
    xxxx     unsigned byte   ??               label
    The labels values are 0 to 9.

    :param idx_ubyte_file: idx文件路径
    :return: n*1维np.array对象，n为图片数量
    """
    return decode_idx1_ubyte(idx_ubyte_file)


def load_test_images(idx_ubyte_file=test_images_idx3_ubyte_file):
    """
    TEST SET IMAGE FILE (t10k-images-idx3-ubyte):
    [offset] [type]          [value]          [description]
    0000     32 bit integer  0x00000803(2051) magic number
    0004     32 bit integer  10000            number of images
    0008     32 bit integer  28               number of rows
    0012     32 bit integer  28               number of columns
    0016     unsigned byte   ??               pixel
    0017     unsigned byte   ??               pixel
    ........
    xxxx     unsigned byte   ??               pixel
    Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black).

    :param idx_ubyte_file: idx文件路径
    :return: n*row*col维np.array对象，n为图片数量
    """
    return decode_idx3_ubyte(idx_ubyte_file)


def load_test_labels(idx_ubyte_file=test_labels_idx1_ubyte_file):
    """
    TEST SET LABEL FILE (t10k-labels-idx1-ubyte):
    [offset] [type]          [value]          [description]
    0000     32 bit integer  0x00000801(2049) magic number (MSB first)
    0004     32 bit integer  10000            number of items
    0008     unsigned byte   ??               label
    0009     unsigned byte   ??               label
    ........
    xxxx     unsigned byte   ??               label
    The labels values are 0 to 9.

    :param idx_ubyte_file: idx文件路径
    :return: n*1维np.array对象，n为图片数量
    """
    return decode_idx1_ubyte(idx_ubyte_file)




train_images = load_train_images()
train_labels = load_train_labels()
test_images = load_test_images()
test_labels = load_test_labels()
final_test_images = load_test_images(final_test_idx3_ubyte_file)

**讓label變成one-hot (ex: 1 ->[0,1,...,0,0])**

In [ ]:
y_train = tf.keras.utils.to_categorical(train_labels)
y_test = tf.keras.utils.to_categorical(test_labels)

**Normalize**
本來image都是介於0~255 透過這動作變成-1~1之間

In [ ]:
x_train = (train_images-127.5)/127.5
x_test = (test_images-127.5)/127.5
final_x = (final_test_images-127.5)/127.5

In [ ]:
#不同的NN需要不同格式輸入
""""x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
final_x = final_x.reshape(-1, 28, 28, 1)

In [ ]:
print("X_train shape after splitting : {}".format(x_train.shape))
print("Shape of input images for Validation after splitting : {}".format(x_test.shape))
print("y_label shape after splitting : {}".format(y_train.shape))
print("Shape of output labels for Validation after splitting : {}".format(y_test.shape))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
""""model = Sequential()

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

In [ ]:
""""#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 0.001), metrics = ['accuracy'])
history = model.fit(x_train, y_train, 
                    epochs = 25, batch_size = 32, 
                    validation_data = (x_test, y_test))

**Define model and train**
opt=adam
loss=crossentropy
epoch=20

In [ ]:
from tensorflow.keras.layers import Dropout
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')    
])

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) #learning_rate mostly adjust *10 or /10
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))



In [ ]:
model.summary()

**Use the trained model to predict on fianl_test images**

In [ ]:
final_y = model.predict(final_x)
final_pred = np.argmax(model.predict(final_x), axis=1)

**Write the prediction in .csv file**

In [ ]:
import csv

# 開啟輸出的 CSV 檔案
with open('output.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

  # 寫入一列資料
    writer.writerow(['ID', 'Class'])
    for i,_ in enumerate(final_pred):
         writer.writerow([i+1,final_pred[i] ])

  # 寫入另外幾列資料

**另一種方法輸出.csv檔**

In [ ]:
id=[]
for i,_ in enumerate(final_pred):
    id.append(i+1)
### final_test has 4563 pictures
print(id)

In [ ]:
results = pd.DataFrame({
    'ID' : id,
    'Class' : final_pred
})

results.to_csv('predict.csv', index=False)